<h1 align="center">Implementing a standard DQN</h1> 

Implementing a basic version of the original DQN. The code was writen based/inspired on the contributions from [Denny Britz](https://github.com/dennybritz/reinforcement-learning/tree/master/DQN) and [Devsisters](https://github.com/devsisters/DQN-tensorflow).

In [1]:
import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import random
import sys
import cv2
import os
import time
from tqdm import tqdm

from inspect import getsourcefile
current_path = os.path.dirname(os.path.abspath(getsourcefile(lambda:0)))
import_path = os.path.abspath(os.path.join(current_path, ".."))

if import_path not in sys.path:
    sys.path.append(import_path)

import commonOps as cops

## Configuration parameters

Configuration parameteres as presented in the [original](http://www.nature.com/nature/journal/v518/n7540/abs/nature14236.html) paper.

In [2]:
conf_parameters = {
    'max_steps': 50000000,
    'eval_freq': 100000,
    'eval_steps': 50000,
    # Input size
    'screen_width': 84,
    'screen_height': 84,
    'history_length': 4,
    'pool_frame_size': 1,
    
    'memory_size': 1000000,       # Replay memory size
    'batch_size': 32,             # Number of training cases ove which SGD update is computed
    'gamma': 0.99,                # Discount factor
    'learning_rate': 0.00025,     # Learning rate
    'learn_start': 50000,         # Replay start size
    'random_start': 30,           # Maximum number of 'do nothing' actions at the start of an episode
    
    # Exploration parameters
    'ep_min': 0.1,                 # Final exploration
    'ep_start': 1.0,               # Initial exploration
    'ep_end_time': 1000000,        # Final exploration frame

    'target_q_update_step': 10000, # Target network update frequency

    # Train batch
    'train_frequency': 4,          # Update frequency
    
    # Clip rewards
    'min_reward': -1.0,
    'max_reward': 1.0,

    # How many times should the same action be taken
    'action_repeat': 1,
    
    # Whether or not to render the environment 
    'display': False,
    
    'checkpoint_dir': 'Models/',
    'log_dir': 'Logs/',
    'log_performance_file': 'Logs/DQN.txt'
}

if not os.path.exists(conf_parameters['checkpoint_dir']):
    os.makedirs(conf_parameters['checkpoint_dir'])
    
if not os.path.exists(conf_parameters['log_dir']):
    os.makedirs(conf_parameters['log_dir'])

## Load the game environment

Wrapper class arund the gym environment. 

In [3]:
class GameScreen:
    def __init__(self, conf):
        self.screens = np.zeros([conf['pool_frame_size'],conf['screen_height'],conf['screen_width']],dtype=np.float32)
        
    def add(self, screen):
        self.screens[:-1] = self.screens[1:]
        self.screens[-1] = screen
        
    def get(self):        
        return np.amax(np.transpose(self.screens,(1,2,0)),axis=2)
    
class GymEnvironment():
    def __init__(self,name,conf):
        self.env = gym.make(name)            # Initialize Gym environment
        self.game_screen = GameScreen(conf)
        self.screen_width = conf['screen_width']         
        self.screen_height = conf['screen_height']
        self.random_start = conf['random_start']
        self.action_repeat = conf['action_repeat']
        self.pool_frame_size = conf['pool_frame_size']
        self.display = conf['display']
        
    def new_game(self):
        self._observation = self.env.reset()
        
        for i in range(self.pool_frame_size):
            self.game_screen.add(self.observation)
            
        self.render()        
        return self.screen
    
    def new_random_game(self):  # Starts a random new game doing 
        _ = self.new_game()
        terminal = False
        for _ in xrange(random.randint(0,self.random_start-1)):
            self._observation, reward, terminal, _ = self.env.step(0)
            self.game_screen.add(self.observation)
        
        self.render()
        return self.screen, 0, terminal
        
    def execute_action(self,action,is_training=True):
        # This function execute the selected action for 'action_repeat' number of times and returns the cumulative reward
        # and final state
        cum_reward = 0
        start_lives = self.num_lives
        
        for _ in xrange(self.action_repeat):
            self._observation, reward, terminal, _ = self.env.step(action)
            self.game_screen.add(self.observation)
            cum_reward += reward
            
            if is_training and start_lives > self.num_lives:
                terminal = True
                
            if terminal:
                break
                
        self.render()
        
        return self.screen, cum_reward, terminal
    
    @property
    def screen(self):
        return self.game_screen.get()
        
    @property
    def action_size(self):
        return self.env.action_space.n        # Number of available actions

    @property
    def num_lives(self):
        return self.env.ale.lives()
    
    @property
    def observation(self):     # Method to resize the screen provided by gym to the desired values
        return cv2.resize(cv2.cvtColor(self._observation,cv2.COLOR_RGB2GRAY)/255.,(self.screen_width,self.screen_height))
    
    def render(self):    # Renders the environment only if display == True
        if self.display:
            self.env.render()  

## Experience Replay
This class will allow us to store the experiences and to take random samples to update our target network parameters.

In [4]:
class Experience_Replay():
    def __init__(self,conf):
        self.memory_size = conf['memory_size']
        # These are the arrays where we will store the experiences
        self._experience = {}
        self.batch_size = conf['batch_size']       
        self.current = 0     # Pointer to the current saving location
        self.count = 0       # Number of collected experiences
        
    def add(self, experience):
        '''
        Stores experience: experience is a tuple of (s1,a,r,s2,t)
        '''        
        if self.current in self._experience:
            del self._experience[self.current]
        self._experience[self.current] = experience
        
        self.count = max(self.count, self.current+1)
        self.current = (self.current + 1) % self.memory_size
                    
    def retrieve(self,indices):
        '''
        Get experiences from indices
        '''
        return [self._experience[v] for v in indices]
        
    def sample_from_replay(self):
        # Randomly sampling from experiences (Standard implementation)
        indexes = []
        while len(indexes) < self.batch_size:
            index = random.randint(0,self.count-1)
            indexes.append(index)
            
        experience_batch = self.retrieve(indexes)
        state, action , reward, next_state, terminal = map(np.array, zip(*experience_batch))
        
        return state, action, reward, next_state, terminal

## History
This class will allow us to stack the last K screens to use them as the input to the network (history of states).

In [5]:
class History:
    def __init__(self, conf):
        self.history = np.zeros([conf['history_length'],conf['screen_height'],conf['screen_width']],dtype=np.float32)
        
    def add(self, screen):
        self.history[:-1] = self.history[1:]
        self.history[-1] = screen
        
    def get(self):
        return np.transpose(self.history,(1,2,0))

## Defining our network architecture

In [7]:
class ConvNet_Estimator():
    '''Q-value estimator neural network
       This architecture will be used both for the Q-network and the Target network.
    '''
    
    def __init__(self,conf,num_actions,scope='estimator',collection=None):
        self.scope = scope
        self.num_actions = num_actions
        self.screen_width = conf['screen_width']
        self.screen_height = conf['screen_height']
        self.history_length = conf['history_length']
        self.ep_min = conf['ep_min']
        self.ep_start = conf['ep_start']
        self.ep_end_time = conf['ep_end_time']
        self.learn_start = conf['learn_start']
        self.batch_size = conf['batch_size']
        self.learning_rate = conf['learning_rate']
        self.gamma = conf['gamma']
        self.collection=collection        

        with tf.variable_scope(scope):
            # Build the graph
            self._build_model()
                
    def _build_model(self):
        '''
        Building the network architecture
        '''
        conv_stack_shape=[(32,8,4),
                    (64,4,2),
                    (64,3,1)]
        
        # Our input: 4(or history_length) Frames taken from the environment
        self.X_ph = tf.placeholder(tf.float32,[None, self.screen_height, self.screen_width, self.history_length],name='X')

        self.head = cops.conv_stack(self.X_ph, conv_stack_shape, self.collection)
        self.head = cops.flatten(self.head)
        self.head = cops.add_relu_layer(self.head, size=512, Collection=self.collection)
        self.Q = cops.add_linear_layer(self.head, self.num_actions, self.collection, layer_name=self.collection+'Q')

        self.best_action = tf.argmax(self.Q,dimension=1)
        
        # Compute loss only for the training network
        if self.collection == 'Normal':            
            # The TD target value
            self.QT_ph = tf.placeholder(tf.float32,[None],name='QT_ph')
            # Integer id of selected action
            self.action_ph = tf.placeholder(tf.int32,[None],name='action_ph')

            self.reward_ph = tf.placeholder(tf.float32, [None], name="reward_ph")
            self.terminal_ph = tf.placeholder(tf.float32, [None], name="terminal_ph")

            with tf.name_scope("loss"):
                # One hot of the action which was taken
                action_one_hot = tf.one_hot(self.action_ph,self.num_actions, 1., 0., name='action_one_hot')
                # Get the prediction of the chosen actions only
                acted_Q = tf.reduce_sum(self.Q * action_one_hot, reduction_indices=1, name='DQN_acted')

                Y = self.reward_ph + self.gamma * self.QT_ph * (1 - self.terminal_ph)
                Y = tf.stop_gradient(Y)

                loss_batch = cops.clipped_l2(Y, acted_Q)
                loss = tf.reduce_sum(loss_batch, name="loss")
        
                self.train_op, grads = cops.graves_rmsprop_optimizer(loss, self.learning_rate, 0.95, 0.01, 1)
        
            # Summaries for Tensorboard
            self.summaries = tf.merge_summary([
                    tf.scalar_summary('loss', loss),
                    tf.scalar_summary('avg_q',tf.reduce_mean(self.Q)),
                    tf.histogram_summary('q_values_hist',self.Q),
                    tf.scalar_summary('max_q_value',tf.reduce_max(self.Q))
                ])      
                
    def predict(self,sess,state):
        '''
        Predicts action values.
        '''
        return sess.run(self.Q,{self.X_ph:state})
    
    def determine_action(self,sess,state,current_step,test_ep=None):
        '''
        Predicts action based on q values for current state and exploration strategy
        '''
        # Calculate exploration prob. epsilon => This is a decaying epsilon starting at 1 and decreasing until 0.1 once the learning process start
        ep = test_ep or (self.ep_min + max(0.,(self.ep_start-self.ep_min)*(self.ep_end_time-max(0.,current_step-self.learn_start))/self.ep_end_time))
        if random.random() < ep:
            # Explore: random action
            action = random.randrange(self.num_actions)
        else:            
            action = sess.run(self.best_action,{self.X_ph:[state]})[0]

        return action
    
    def update(self,sess,state,action,reward,target,terminal):
        '''
        Updates the estimator towards the given targets
        '''
        feed_dict = {self.X_ph:state, self.QT_ph:target, 
                     self.action_ph:action, self.reward_ph:reward,
                     self.terminal_ph:terminal}
        _, summaries = sess.run([self.train_op, self.summaries],feed_dict)

        return summaries

## Learning agent

In [8]:
class Agent():
    def __init__(self,conf,game_name,sess):
        self.conf = conf
        self.sess = sess
        self.env = GymEnvironment(game_name,conf)
        self.exp_replay = Experience_Replay(self.conf)
        self.history = History(self.conf)
        self.checkpoint_dir = conf['checkpoint_dir']
        self.log_performance_file = conf['log_performance_file']
        self.log_dir = conf['log_dir']
        self.history_length = conf['history_length']
        self.learn_start = conf['learn_start']  
        self.train_frequency = conf['train_frequency']
        self.target_q_update_step = conf['target_q_update_step']
        self.eval_freq = conf['eval_freq']
        self.eval_steps = conf['eval_steps']
        self.ep_min = conf['ep_min']
        self.max_steps = conf['max_steps']
        self.min_reward = conf['min_reward']
        self.max_reward = conf['max_reward']
        
        self.eval_env = GymEnvironment(game_name,conf)
        self.eval_history = History(conf)
        
        self.q_estimator = ConvNet_Estimator(conf_parameters,self.env.action_size,scope='q',collection='Normal')
        self.target_estimator = ConvNet_Estimator(conf_parameters,self.env.action_size,scope='target_q',collection='Target')
        
        with tf.variable_scope('step'):
            self.step_op = tf.Variable(0,trainable=False,name='step')
            self.step_input = tf.placeholder(tf.int32,None,name='step_input')
            self.step_assign_op = self.step_op.assign(self.step_input)
            
        self.sync_QT_op = []
        for W_pair in zip(tf.get_collection("Target_weights"),tf.get_collection("Normal_weights")):
            self.sync_QT_op.append(W_pair[0].assign(W_pair[1]))
        
        # Add summaries
        with tf.variable_scope('summary'):
            scalar_summary_tags = ['average.reward', 'episode.max reward', 'episode.min reward',\
                                  'episode.avg reward', 'episode.num of games']
            self.summary_placeholders = {}
            self.summary_ops = {}
            for tag in scalar_summary_tags:
                self.summary_placeholders[tag] = tf.placeholder(tf.float32,None,name=tag.replace(' ','_'))
                self.summary_ops[tag] = tf.scalar_summary(tag, self.summary_placeholders[tag])
            
            histogram_summary_tags = ['episode.rewards']
            for tag in histogram_summary_tags:
                self.summary_placeholders[tag] = tf.placeholder(tf.float32,None,name=tag.replace(' ','_'))
                self.summary_ops[tag] = tf.histogram_summary(tag, self.summary_placeholders[tag])
                
            self.writer = tf.train.SummaryWriter(self.log_dir,self.sess.graph)
        
        tf.initialize_all_variables().run()
        self.saver = tf.train.Saver(max_to_keep = 3)
        
        self.load_model()
        self.update_target_q_network()

    def update_target_q_network(self):
        self.sess.run(self.sync_QT_op)
            
    def load_model(self,step=None):
        latest_checkpoint = tf.train.latest_checkpoint(self.checkpoint_dir)
        if latest_checkpoint:
            print('[*] Loading Checkpoint {}...\n'.format(latest_checkpoint))
            self.saver.restore(self.sess,latest_checkpoint)
        else:
            print('[!] Could not find any previous checkpoint')
        
    def save_model(self,step=None):
        f = open(self.log_performance_file, 'a')
        f.write('[*] Saving checkpoints...')
        f.close()

        self.saver.save(self.sess,os.path.join(self.checkpoint_dir,'model'),global_step=step)    
        
    def train(self):
        start_step = self.step_op.eval()
        max_avg_ep_reward = 0.
        screen,_,terminal = self.env.new_random_game()
        
        # Stacking the screen in the first input buffer
        for _ in range(self.history_length):
            self.history.add(screen)
            
        # Training:
        for self.step in tqdm(range(start_step,self.max_steps),ncols=70,initial=start_step):               
            # 1. Predict: Use our training network to select an action
            action = self.q_estimator.determine_action(self.sess,self.history.get(),self.step)
            
            # 2. Execute the action
            screen, reward, terminal = self.env.execute_action(action,is_training=True)
            
            # 3. New observation
            self.observe(screen,reward,action,terminal)
            
            if terminal:
                screen,_,terminal = self.env.new_random_game()
                for _ in range(self.history_length):
                    self.history.add(screen)
                
            if self.step >= self.learn_start and self.step % self.eval_freq == 0:
                # New game
                print('Evaluating...\n')
                screen = self.eval_env.new_game()
                for _ in range(self.history_length):
                    self.eval_history.add(screen)
                # 
                total_reward, ep_reward = 0., 0.
                n_rewards = 0
                n_episodes = 0
                ep_rewards = []
                
                for eval_step in range(self.eval_steps):
                    # Take action with exploration e= 0.05
                    action = self.q_estimator.determine_action(self.sess,self.eval_history.get(),0,test_ep=0.05)
                    # Play game in test mode (Episodes do not end when losing a life)
                    screen, reward, terminal = self.eval_env.execute_action(action,is_training=False)
                    self.eval_history.add(screen)
                    # Clip reward
                    # reward = max(self.min_reward,min(self.max_reward,reward))
                    # Record every reward
                    ep_reward += reward
                    #if reward != 0:
                    n_rewards += 1
                    
                    if terminal:
                        ep_rewards.append(ep_reward)
                        total_reward += ep_reward
                        ep_reward = 0
                        n_episodes += 1
                        screen,_,terminal = self.eval_env.new_random_game()
                        for _ in range(self.history_length):
                            self.eval_history.add(screen)
                                                                
                #q_t,avg_loss,summaries = self.q_learning_mini_batch()
                
                avg_reward = total_reward / float(max(n_rewards,1))

                try:
                    max_ep_reward = np.amax(ep_rewards)
                    min_ep_reward = np.amin(ep_rewards)
                    avg_ep_reward = np.mean(ep_rewards)
                except:
                    max_ep_reward, min_ep_reward, avg_ep_reward = 0.,0.,0.
                
                f = open(self.log_performance_file, 'a')
                f.write('\nAvg. reward: %.4f' % avg_reward)
                f.write(' Avg. Ep. Reward: %.4f, Max Ep. Reward: %.4f, Min Ep. Reward: %.4f, # Game: %d' \
                        % (avg_ep_reward, max_ep_reward, min_ep_reward, n_episodes))
                f.close()

                if max_avg_ep_reward * 1.05 <= avg_ep_reward:
                    # There is at least a 5% improvement 
                    self.step_assign_op.eval({self.step_input: self.step+1})
                    self.save_model(self.step+1)
                    max_avg_ep_reward = avg_ep_reward
                elif self.step % (self.eval_freq * 5) == 0:
                    self.step_assign_op.eval({self.step_input: self.step+1})
                    self.save_model(self.step+1)

                self.inject_summary({
                        'average.reward' : avg_reward,
                        'episode.max reward':max_ep_reward,
                        'episode.min reward':min_ep_reward,
                        'episode.avg reward':avg_ep_reward,
                        'episode.num of games':n_episodes,
                        'episode.rewards': ep_rewards,
                    })
    
    def observe(self,screen,reward,action,terminal):
        # Clip reward
        reward = max(self.min_reward,min(self.max_reward,reward))
        
        prev_state = self.history.get()
        # Add to history
        self.history.add(screen)
        # Add to exp. replay
        self.exp_replay.add((prev_state,reward,action,self.history.get(),terminal))
        
        if self.step > self.learn_start:
            # If it is time to train the network
            if self.step % self.train_frequency == 0:
                _,_,summaries = self.q_learning_mini_batch()
                self.writer.add_summary(summaries,self.step)
                
            # If it is time to update Target network
            if self.step % self.target_q_update_step == self.target_q_update_step -1:
                self.update_target_q_network(self.sess)
        
    def q_learning_mini_batch(self):
        init_state, action, reward, end_state, terminal = self.exp_replay.sample_from_replay()

        # *** NEW: DOUBLE Q-LEARNING!! ***
        best_action_t_plus_1 = self.q_estimator.get_best_action(self.sess,end_state)
        q_t_plus_1_with_pred_action = self.target_estimator.get_output_with_idx(
            self.sess,end_state,[[idx,pred_a] for idx,pred_a in enumerate(best_action_t_plus_1)])
        terminal = np.array(terminal) + 0.
        target_q_t = (1. - terminal) * self.gamma * q_t_plus_1_with_pred_action + reward
        # *** END OF NEW SEGMENT

        q_t, loss, summaries = self.q_estimator.update(self.sess,init_state,action,target_q_t)

        return q_t, loss, summaries       
            
    def inject_summary(self,tag_dir):
        summary_str_lists = self.sess.run([self.summary_ops[tag] for tag in tag_dir.keys()], {
                self.summary_placeholders[tag]: value for tag , value in tag_dir.items()
            })
        for summary_str in summary_str_lists:
            self.writer.add_summary(summary_str,self.step)
            
    def play(self,n_step=10000,n_episode=100,test_epsilon=None):
        if test_epsilon == None:
            test_epsilon = ep_min
        
        test_history = History()
        best_reward, best_idx = 0, 0
        for idx in xrange(n_episode):
            screen,_,_,_ = self.env.new_random_game()
            current_reward = 0
            
            for _ in range(history_length):
                test_history.add(screen)
                
            for t in tqdm(range(n_step),ncols=70):
                # 1. Predict
                action = self.q_estimator.determine_action(self.sess,test_history.get(),0,test_epsilon)
            
                # 2. Execute the action
                screen, reward, terminal = self.env.execute_action(action,is_training=False)                
                
                # 3. New observation
                test_history.add(screen)
                
                current_reward += reward
                if terminal:
                    break
            
            if current_reward > best_reward:
                best_reward = current_reward
                best_idx = idx
                
            print('='*30)
            print('[%d] Best reward: %d' % (best_idx, best_reward))
            print('='*30)
            
        self.env.env.render(close=True)     

## Main code:

In [ ]:
is_train = True
game_name = 'Breakout-v0'

sess_config = tf.ConfigProto()
sess_config.allow_soft_placement = True
sess_config.gpu_options.allow_growth = True
sess_config.log_device_placement = False

with tf.Session(config=sess_config) as sess:
    agent = Agent(conf_parameters,game_name,sess)    
    
    if is_train:
        display = False
        agent.train()
    else:
        display = False
        agent.play()

## Test on CPU

In [ ]:
is_train = True

with tf.Session() as sess:
    env = GymEnvironment('Breakout-v0')
    agent = Agent(env,sess)
    
    if is_train:
        display = False
        agent.train()
    else:
        display = True
        agent.play()